# Pre-Lab 2: Internet Routing

### Configure environment

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager() 
conf = fablib.show_config()

### Define configuration for this experiment (two hosts and router in line topology)

In [ ]:
slice_name="cs2520-lab2-bgp_" + fablib.get_bastion_username()

node_conf = [
 {'name': "perlman",   'cores': 1, 'ram': 4, 'disk': 10, 'image': 'default_ubuntu_22', 'packages': ['whois']},
 {'name': "floyd",   'cores': 1, 'ram': 4, 'disk': 10, 'image': 'default_ubuntu_22', 'packages': ['whois']},
 {'name': "borg",   'cores': 1, 'ram': 4, 'disk': 10, 'image': 'default_ubuntu_22', 'packages': ['whois']},
 {'name': "kahn",   'cores': 1, 'ram': 4, 'disk': 10, 'image': 'default_ubuntu_22', 'packages': ['whois']},
]

### Reserve resources

Now, we are ready to reserve resources!

First, make sure you don’t already have a slice with this name:

In [ ]:
try:
    slice = fablib.get_slice(slice_name)
    print("You already have a slice by this name!")
    print("If you previously reserved resources, skip to the 'log in to resources' section.")
except:
    print("You don't have a slice named %s yet." % slice_name)
    print("Continue to the next step to make one.")
    slice = fablib.new_slice(name=slice_name)

We will reserve resources for our experiment at the four random, geographically spread-apart, sites:

In [ ]:
import random

europe_sites_list = ['AMST', 'CERN', 'BRIST']
pacific_sites_list = ['HAWI', 'TOKY']
non_contUS_sites = europe_sites_list + pacific_sites_list

# Get two sites in different geographic locations, one west of St. Louis, and one east of St. Louis
st_louis_lat_long=(32.773081, -96.797448)
while True:
    west_site = fablib.get_random_site(filter_function=lambda x: x['nic_basic_available'] > 0 and x['location'][1] < st_louis_lat_long[1])                                                                                                                                                                                                                           
    if west_site not in non_contUS_sites:
        break
while True:
    east_site = fablib.get_random_site(filter_function=lambda x: x['nic_basic_available'] > 0 and x['location'][1] > st_louis_lat_long[1])                                                                                                                                                                                                                           
    if east_site not in non_contUS_sites:
        break
        
# Get one site in Europe and choose one from either University of Tokyo or University of Hawaii
europe_site = random.choice(europe_sites_list)
pacific_site = random.choice(pacific_sites_list)

print(f"west_site: {west_site}")
print(f"east_site: {east_site}")
print(f"europe_site: {europe_site}")
print(f"pacific_site: {pacific_site}")

Then we will add hosts and network segments:

In [ ]:
# this cell sets up the nodes
nodes_w_sites_names = zip([west_site, east_site, europe_site, pacific_site], node_conf)
for this_site_name, n in nodes_w_sites_names:
    slice.add_node(name=n['name'], site=this_site_name, 
                   cores=n['cores'], 
                   ram=n['ram'], 
                   disk=n['disk'], 
                   image=n['image'])

The following cell submits our request to the FABRIC site. The output of this cell will update automatically as the status of our request changes.

-   While it is being prepared, the “State” of the slice will appear as “Configuring”.
-   When it is ready, the “State” of the slice will change to “StableOK”.

You may prefer to walk away and come back in a few minutes (for simple slices) or a few tens of minutes (for more complicated slices with many resources).

In [ ]:
slice.submit()

In [ ]:
slice.get_state()
slice.wait_ssh(progress=True)

### Configure resources

Next, we will configure the resources so they are ready to use.

In [ ]:
slice = fablib.get_slice(name=slice_name)

In [ ]:
# install packages
# this will take a while and will run in background while you do other steps
for n in node_conf:
    if len(n['packages']):
        node = slice.get_node(n['name'])
        pkg = " ".join(n['packages'])
        node.execute_thread("sudo apt update; sudo DEBIAN_FRONTEND=noninteractive apt -y install %s" % pkg)

In [ ]:
# make sure all interfaces are brought up
for iface in slice.get_interfaces():
    iface.ip_link_up()

In [ ]:
# turn off segmentation offload on interfaces
for iface in slice.get_interfaces():
    iface_name = iface.get_device_name()
    n = iface.get_node()
    offloads = ["gro", "lro", "gso", "tso"]
    for offload in offloads:
        n.execute("sudo ethtool -K %s %s off" % (iface_name, offload))

### Draw the network topology

The following cell will draw the network topology, for your reference. The interface name and addresses of each experiment interface will be shown on the drawing.

In [ ]:
l2_nets = [(n.get_name(), {'color': 'lavender'}) for n in slice.get_l2networks() ]
l3_nets = [(n.get_name(), {'color': 'pink'}) for n in slice.get_l3networks() ]
hosts   =   [(n.get_name(), {'color': 'lightblue'}) for n in slice.get_nodes()]
nodes = l2_nets + l3_nets + hosts
ifaces = [iface.toDict() for iface in slice.get_interfaces()]
edges = [(iface['network'], iface['node'], 
          {'label': iface['physical_dev'] + '\n' + iface['ip_addr'] + '\n' + iface['mac']}) for iface in ifaces]

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
plt.figure(figsize=(len(nodes),len(nodes)))
G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)
pos = nx.spring_layout(G)
nx.draw(G, pos, node_shape='s',  
        node_color=[n[1]['color'] for n in nodes], 
        node_size=[len(n[0])*400 for n in nodes],  
        with_labels=True);
nx.draw_networkx_edge_labels(G,pos,
                             edge_labels=nx.get_edge_attributes(G,'label'),
                             font_color='gray',  font_size=8, rotate=False);

### Log into resources

Now, we are finally ready to log in to our resources over SSH! Run the following cells, and observe the table output - you will see an SSH command for each of the resources in your topology.

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
slice_info = [{'Name': n.get_name(), 'SSH command': n.get_ssh_command()} for n in slice.get_nodes()]
pd.DataFrame(slice_info).set_index('Name')

Now, you can open an SSH session on any of the resources as follows:

-   in Jupyter, from the menu bar, use File \> New \> Terminal to open a new terminal.
-   copy an SSH command from the table, and paste it into the terminal. (Note that each SSH command is a single line, even if the display wraps the text to a second line! When you copy and paste it, paste it all together.)

You can repeat this process (open several terminals) to start a session on each resource. Each terminal session will have a tab in the Jupyter environment, so that you can easily switch between them.

### Extend your slice

The following cell extends your reservation for 3 days.

In [ ]:
from datetime import datetime
from datetime import timezone
from datetime import timedelta

# Set end date to 3 days from now
end_date = (datetime.now(timezone.utc) + timedelta(days=3)).strftime("%Y-%m-%d %H:%M:%S %z")
slice.renew(end_date)

Confirm the new end time of your slice in the output of the following cell:

In [ ]:
slice.update()
_ = slice.show()

If you need more time, you can extend your slice again anytime before these 3 days have elapsed, .

### Acknowledgment
Adapted from [Faida Fund's](https://witestlab.poly.edu/blog/a-peek-into-internet-routing/)'s experiments.